In [2]:
import os
import pathlib
import re

import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from astroquery.gaia import Gaia
Gaia.login(**snakemake.config["gaia"])

In [3]:
data_dir = pathlib.Path(snakemake.input[0]).parent

In [4]:
username = Gaia._TapPlus__user
sb9_tblname = 'sb9_2021_02'
sb9_archive_tblname = f'user_{username}.{sb9_tblname}'

In [5]:
# see: !head -n 20 SB9public/ReadMe.txt
colnames = [
    'id',
    'sb9_j1900_str',
    'sb9_j2000_str',
    'sb9_component',
    'sb9_mag1',
    'sb9_filter1',
    'sb9_mag2',
    'sb9_filter2',
    'sb9_spt1',
    'sb9_spt2'
]

sb9_tbl = at.Table.read(data_dir / 'SB9public' / 'Main.dta',
                        delimiter='|', format='ascii.basic',
                        names=colnames)

# only keep sources with magnitudes in SB9 (this removes ~90 sources)
sb9_tbl = sb9_tbl[~sb9_tbl['sb9_mag1'].mask]

# also remove whack values - this removes another 2 sources
mag_mask = np.ones(len(sb9_tbl), dtype=bool)
for i, m in enumerate(sb9_tbl['sb9_mag1']):
    try:
        float(m)
        mag_mask[i] = False
    except:
        pass
sb9_tbl = sb9_tbl[~mag_mask]
sb9_tbl['sb9_mag1'] = sb9_tbl['sb9_mag1'].filled(np.nan).astype(float)
sb9_tbl['sb9_mag2'] = sb9_tbl['sb9_mag2'].filled(1e5)  # MAGIC NUMBER

Cool I love parsing string coordinates

In [6]:
pattr = re.compile('([0-9]{2})([0-9]{2})([0-9]{2})([0-9]{1,})([\+\-])([0-9]{2})([0-9]{2})([0-9]{2})([0-9]{0,})')
ra = []
dec = []
for str_c in sb9_tbl['sb9_j2000_str']:
    (ra_h, ra_m, ra_s, ra_s2, 
     dec_sign, dec_d, dec_m, dec_s, dec_s2) = pattr.search(str_c).groups()
    
    ra_hms = float(ra_h) + float(ra_m)/60 + float(f"{ra_s}.{ra_s2}")/3600
    dec_dms = float(dec_d) + float(dec_m)/60 + float(f"{dec_s}.{dec_s2}")/3600
    dec_dms = float(f'{dec_sign}1') * dec_dms
    
    ra.append(ra_hms * u.hourangle)
    dec.append(dec_dms * u.degree)

# simply validation by passing in to lon/lat classes:
ra = coord.Longitude(ra)
dec = coord.Latitude(dec)

sb9_tbl['ra_deg'] = ra.degree
sb9_tbl['dec_deg'] = dec.degree

In [7]:
tblnames = [t.name for t in Gaia.load_tables(only_names=True)]

if sb9_archive_tblname not in tblnames:
    job = Gaia.upload_table(sb9_tbl['id', 'ra_deg', 'dec_deg'], 
                            table_name=sb9_tblname)

For subqueries, we can't use a wildcard to get all column names, so we explicitly have to ask for all gaia columns...

In [8]:
gaia_tbl = Gaia.load_table('gaiaedr3.gaia_source')
all_gaia_columns = [f"subq.{c.name}" for c in gaia_tbl.columns]

In [9]:
init_tol = 30 * u.arcsec
final_tol = 4 * u.arcsec
sb9_epoch = 2000.  # This is a guess!
 
q = f"""
SELECT subq.id, {', '.join(all_gaia_columns)}
FROM (
    SELECT sb9.id, sb9.ra_deg, sb9.dec_deg, gaia.*
    FROM {sb9_archive_tblname} AS sb9, gaiaedr3.gaia_source as gaia
    WHERE 
        contains(POINT('ICRS', sb9.ra_deg, sb9.dec_deg),
                 CIRCLE('ICRS', gaia.ra, gaia.dec, {init_tol.to_value(u.deg)}))=1
    OFFSET 0
) AS subq
WHERE 
    contains(POINT('ICRS', subq.ra + subq.pmra / 3600e3  * ({sb9_epoch} - subq.ref_epoch) / COS(RADIANS(subq.dec)), 
                           subq.dec + subq.pmdec / 3600e3 * ({sb9_epoch} - subq.ref_epoch)),
             CIRCLE('ICRS', subq.ra_deg, subq.dec_deg, {final_tol.to_value(u.deg)}))=1
"""

In [10]:
job = Gaia.launch_job_async(q, name='sb9-edr3')

In [11]:
xm_tbl = job.get_results()

In [12]:
joined = at.join(sb9_tbl, xm_tbl, keys='id')

In [13]:
mag = -np.log10((10**-joined['sb9_mag1'] + 10**-joined['sb9_mag2']))

plt.figure(figsize=(6, 6))
plt.scatter(joined['phot_g_mean_mag'], mag - joined['phot_g_mean_mag'],
            alpha=0.5, lw=0, s=4)
plt.xlabel('$G$ [mag]')
plt.ylabel(r'$V_{\rm SB9} - G$ [mag]')
plt.ylim(-5, 5)

OK so there are definitely sources with unexpected Gaia mags given the V-band mags in SB9, so we can trim those as (hopefully) duplicate / spurious matches. We could do something much better here, since there is a color-dependent transform between G <--> V...but, ya'know, hacks:

In [14]:
dmag = mag - joined['phot_g_mean_mag']

In [15]:
dmag_cut = 0.5

plt.figure(figsize=(6, 6))
plt.scatter(joined['phot_g_mean_mag'], 
            dmag - np.median(dmag),
            alpha=0.5, lw=0, s=4)
plt.xlabel('$G$ [mag]')
plt.ylabel(r'$V_{\rm SB9} - G$ [mag]')
plt.ylim(-5, 5)
plt.axhline(dmag_cut)
plt.axhline(-dmag_cut)

In [16]:
joined_dmag = joined[np.abs(dmag - np.median(dmag)) < dmag_cut]

In [17]:
_, counts = np.unique(joined_dmag['id'], return_counts=True)
(counts > 1).sum()

In [18]:
len(joined_dmag)

Now let's just save the ones with measured RV error from Gaia:

In [19]:
joined_dmag_with_rv = joined_dmag[np.isfinite(joined_dmag["dr2_radial_velocity_error"])]

In [20]:
len(joined_dmag_with_rv)

In [21]:
plt.hist(joined_dmag_with_rv["dr2_radial_velocity_error"], 50)
plt.xlabel("Gaia DR2 RV error")
plt.ylabel("count");

In [22]:
plt.hist(joined_dmag_with_rv["dr2_rv_nb_transits"], 50)
plt.xlabel("Gaia number of RVS transits")
plt.ylabel("count");

In [23]:
colnames = [
    "id",
    "sb9_number",
    "sb9_period",
    "sb9_period_err",
    "sb9_t_peri",
    "sb9_t_peri_err",
    "sb9_t_peri_flag",
    "sb9_eccen",
    "sb9_eccen_err",
    "sb9_omega",
    "sb9_omega_err",
    "sb9_k1",
    "sb9_k1_err",
    "sb9_k2",
    "sb9_k2_err",
    "sb9_rv0",
    "sb9_rv0_err",
    "sb9_rv_rms1",
    "sb9_rv_rms2",
    "sb9_num_rv1",
    "sb9_num_rv2",
    "sb9_grade",
    "sb9_bibcode",
    "sb9_contributor",
    "sb9_accessibility",
    "sb9_time_ref",
]
    
orbit_tbl = at.Table.read(data_dir / 'SB9public' / 'Orbits.dta',
                          delimiter='|', format='ascii.basic',
                          names=colnames)

In [24]:
final = at.join(joined_dmag_with_rv, orbit_tbl, keys=["id"])
del final["designation"]

k1 = final["sb9_k1"]
k1[k1 == "12.D0"] = "12.0"
final["sb9_k1"] = k1.filled(np.nan).astype(float)

In [25]:
plt.loglog(final["sb9_k1"], final["dr2_radial_velocity_error"], ".")
plt.xlabel("SB9 K1")
plt.ylabel("Gaia DR2 RV error")

In [26]:
final.write(snakemake.output[0], format='fits', overwrite=True)